To begin to develop the `kinisi` code, you should clone the git repository and install `kinisi` with the `dev` option in editable mode. 

```console
$ git clone https://github.com/bjmorgan/kinisi.git
$ cd kinisi
$ pip install -e '.[dev]'
```

**When developing, please follow this style guide: https://llego.dev/posts/writing-clean-pep-8-compliant-code-better-collaboration/**

Below a brief over view of the kinisi codebase is given to aid the onboarding of new developers.

# DiffusionAnalyzer and JumpDiffusionAnalyzer

Within `kinisi` the main classes the end user will interact with are [`DiffusionAnalyzer`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L20), [`JumpDiffusionAnalyzer`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/jump_diffusion_analyzer.py#L20) and [`ConductivityAnalyzer`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/conductivity_analyzer.py#L20)

These are typically called with 3 main methods depending on the type of trajectory these are:

* [`from_universe`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L87) - For MDAnalysis Universe
* [`from_xdatcar`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L35) - For Pymatgen Structures
* [`from_ase`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py) - For ASE objects

For instance, [`DiffusionAnalyzer.from_xdatcar`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L35) can be called via

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import scipp as sc
from pymatgen.io.vasp import Xdatcar
from kinisi.analyze import DiffusionAnalyzer
rng = np.random.RandomState(42)

params = {'specie': 'Li',
          'time_step': 2.0 * sc.Unit('fs'),
          'step_skip': 50 * sc.Unit('dimensionless'),
          }


params['dimension']= 'xy'

xd = Xdatcar('./example_XDATCAR.gz')
diff = DiffusionAnalyzer.from_xdatcar(xd, **params)

Finding Means and Variances: 100%|██████████| 140/140 [00:00<00:00, 2160.21it/s]


[`DiffusionAnalyzer`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L20) is a subclass of [`Analyzer`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/analyzer.py#L20) and so upon its initialization `Analyzer` is initialized first, `DiffusionAnalyzer` inherits from the `Analyzer`.

```python

class DiffusionAnalyzer(Analyzer):

```


The `from_` class method of the `DiffusionAnalyzer` calls the Analyzer class `_from_` method via a super(), i.e. within [`from_universe`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L131)

```python

p = super()._from_universe(trajectory, specie, time_step, step_skip, dtype, dt, dimension, distance_unit,
                            specie_indices, masses, progress)

```

Within the `_from_` methods the respective parser is called for each frame in the trajectory, similarly to above there are 3 parser classes : 


* [`MDAnalysis_parser`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/analyzer.py#L109) - For MDAnalysis Universe
* [`PymatgenParser`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/analyzer.py#L74) - For Pymatgen Structures
* [`ASE_parser`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/analyzer.py) - For ASE objects


Each respective parser class is a subclass of the full [`Parser`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/parser.py#L34) class. 

```python 

class PymatgenParser(Parser):

```

**The parser indexes the correct molecules and particle coordinates for a timestep.**

**The Analyzer calls the parser for all the chosen time steps and calculates displacements.**

**The DiffusionAnalyzer then calculates the mean squared displacements (MSD) and variances for each time interval.**

A flowchart of the classes and inheritance can be seen below



![My Diagram](./diffusion_analyzer_flowchart.png)

The main output of the `from_` methods is then [`.msd_da`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L83) which is a [Scipp](https://scipp.github.io/) DataArray containing the MSD, n_samples, time intervals as well as standard deviations, it can be accessed via

In [6]:
diff.msd_da

<scipp.DataArray>
Dimensions: Sizes[time interval:140, ]
Coordinates:
* dimensionality              int64  [dimensionless]  ()  2
* n_samples                 float64  [dimensionless]  (time interval)  [26880, 13440, ..., 193.381, 192]
* time interval             float64             [fs]  (time interval)  [100, 200, ..., 13900, 14000]
Data:
                            float64           [Å^2]  (time interval)  [0.29172, 0.464273, ..., 7.59248, 7.73364]  [5.20148e-06, 2.72036e-05, ..., 0.547806, 0.560402]

# DiffusionAnalyzer.diffusion()

In order to calculate the diffusion coefficient posterior distribution, the [`.diffusion`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion_analyzer.py#L136) function is called with a specified `start_dt`

In [7]:
start_of_diffusion = 3000 * sc.Unit('fs')
diff.diffusion(start_of_diffusion, progress=False, random_state=rng)

This method initializes the [`Diffusion`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L20) class and calls the [`_diffusion`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L131) method on the initalized class. 

Within `_diffusion` the bayesian regression model is produced via MCMC sampling within [`bayesian_regression`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L46). 

This involves the calculation of the full covariance matrix with [`compute_covariance_matrix`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L191), matrix conditioning with [`minimum_eigenvalue_method`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L257) and then MCMC posterior prediction via the [emcee module](https://emcee.readthedocs.io/en/stable/) and [`posterior_predictive`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L209) function.

For more information on the bayesian framework used in kinisi, please see the [methodology section](https://kinisi.readthedocs.io/en/latest/methodology.html) of the kinisi documentation.

The sampled posterior distribution can then be accessed via the [`.D`](https://github.com/kinisi-dev/kinisi/blob/scipp/kinisi/diffusion.py#L171) property

In [8]:
diff.D

<scipp.Variable> (samples: 3200)    float64         [cm^2/s]  [1.24188e-05, 1.22728e-05, ..., 1.40545e-05, 1.33546e-05]

A flowchart of the classes and main calls can be seen below

![My Diagram](./diffusion_flowchart.png)

That concludes the brief run down of the kinisi code base, if you would like to get involved with development, please see the issues list on the GitHub repo, or feel free to raise your own or reach out to one of the current developers. 

Happy developing :) 
